# Importando e preparando os dados

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


/home/marco/dl_env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/marco/dl_env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Avaliando

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [15]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

- Capturando média e desvio padrão da acurácia

In [22]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(accuracies.mean())
print(accuracies.std())

0.8271249962970615
0.01134749438249182


## Encontrando os melhores parâmetros

In [29]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [30]:
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [5, 10, 50],
              'optimizer': ['sgd', 'adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [31]:
grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
7200/7200 [==============================] - 1s 81us/step - loss: 0.6097 - acc: 0.7969
Epoch 2/5
7200/7200 [==============================] - 0s 53us/step - loss: 0.5353 - acc: 0.7971
Epoch 3/5
7200/7200 [==============================] - 0s 47us/step - loss: 0.5146 - acc: 0.7971
Epoch 4/5
7200/7200 [==============================] - 0s 52us/step - loss: 0.5080 - acc: 0.7971
Epoch 5/5
7200/7200 [==============================] - 0s 49us/step - loss: 0.5057 - acc: 0.7971
Epoch 1/5
7200/7200 [==============================] - 1s 80us/step - loss: 0.6103 - acc: 0.7949
Epoch 2/5
7200/7200 [==============================] - 0s 56us/step - loss: 0.5360 - acc: 0.7967
Epoch 3/5
7200/7200 [==============================] - 0s 48us/step - loss: 0.5152 - a

7200/7200 [==============================] - 0s 57us/step - loss: 0.5262 - acc: 0.7938
Epoch 1/5
7200/7200 [==============================] - 1s 191us/step - loss: 0.5612 - acc: 0.7947
Epoch 2/5
7200/7200 [==============================] - 0s 59us/step - loss: 0.4431 - acc: 0.7944
Epoch 3/5
7200/7200 [==============================] - 0s 58us/step - loss: 0.4351 - acc: 0.7944
Epoch 4/5
7200/7200 [==============================] - 0s 57us/step - loss: 0.4358 - acc: 0.7944
Epoch 5/5
7200/7200 [==============================] - 0s 59us/step - loss: 0.4325 - acc: 0.7972
Epoch 1/5
7200/7200 [==============================] - 1s 199us/step - loss: 0.5430 - acc: 0.7962
Epoch 2/5
7200/7200 [==============================] - 0s 60us/step - loss: 0.4424 - acc: 0.7969
Epoch 3/5
7200/7200 [==============================] - 0s 60us/step - loss: 0.4380 - acc: 0.7969
Epoch 4/5
7200/7200 [==============================] - 0s 59us/step - loss: 0.4336 - acc: 0.7969
Epoch 5/5
7200/7200 [=================

7200/7200 [==============================] - 0s 62us/step - loss: 0.5045 - acc: 0.7971
Epoch 10/10
7200/7200 [==============================] - 0s 64us/step - loss: 0.5045 - acc: 0.7971
Epoch 1/10
7200/7200 [==============================] - 2s 311us/step - loss: 0.6100 - acc: 0.7940
Epoch 2/10
7200/7200 [==============================] - 0s 67us/step - loss: 0.5357 - acc: 0.7967
Epoch 3/10
7200/7200 [==============================] - 0s 66us/step - loss: 0.5151 - acc: 0.7967
Epoch 4/10
7200/7200 [==============================] - 0s 66us/step - loss: 0.5086 - acc: 0.7967
Epoch 5/10
7200/7200 [==============================] - 0s 66us/step - loss: 0.5064 - acc: 0.7967
Epoch 6/10
7200/7200 [==============================] - 0s 66us/step - loss: 0.5055 - acc: 0.7967
Epoch 7/10
7200/7200 [==============================] - 0s 66us/step - loss: 0.5052 - acc: 0.7967
Epoch 8/10
7200/7200 [==============================] - 0s 66us/step - loss: 0.5051 - acc: 0.7967
Epoch 9/10
7200/7200 [=======

7200/7200 [==============================] - 1s 70us/step - loss: 0.5370 - acc: 0.7961
Epoch 3/10
7200/7200 [==============================] - 0s 69us/step - loss: 0.5161 - acc: 0.7961
Epoch 4/10
7200/7200 [==============================] - 0s 68us/step - loss: 0.5094 - acc: 0.7961
Epoch 5/10
7200/7200 [==============================] - 0s 69us/step - loss: 0.5071 - acc: 0.7961
Epoch 6/10
7200/7200 [==============================] - 0s 69us/step - loss: 0.5063 - acc: 0.7961
Epoch 7/10
7200/7200 [==============================] - 0s 68us/step - loss: 0.5060 - acc: 0.7961
Epoch 8/10
7200/7200 [==============================] - 0s 69us/step - loss: 0.5058 - acc: 0.7961
Epoch 9/10
7200/7200 [==============================] - 0s 69us/step - loss: 0.5058 - acc: 0.7961
Epoch 10/10
7200/7200 [==============================] - 0s 68us/step - loss: 0.5058 - acc: 0.7961
Epoch 1/10
7200/7200 [==============================] - 3s 397us/step - loss: 0.5591 - acc: 0.7957
Epoch 2/10
7200/7200 [=======

7200/7200 [==============================] - 1s 89us/step - loss: 0.4352 - acc: 0.7962
Epoch 6/10
7200/7200 [==============================] - 1s 86us/step - loss: 0.4302 - acc: 0.7963
Epoch 7/10
7200/7200 [==============================] - 1s 87us/step - loss: 0.4307 - acc: 0.7963
Epoch 8/10
7200/7200 [==============================] - 1s 88us/step - loss: 0.4302 - acc: 0.7962
Epoch 9/10
7200/7200 [==============================] - 1s 85us/step - loss: 0.4292 - acc: 0.7962
Epoch 10/10
7200/7200 [==============================] - 1s 87us/step - loss: 0.4271 - acc: 0.7962
Epoch 1/10
7200/7200 [==============================] - 4s 487us/step - loss: 0.5747 - acc: 0.7947
Epoch 2/10
7200/7200 [==============================] - 1s 84us/step - loss: 0.4435 - acc: 0.7957
Epoch 3/10
7200/7200 [==============================] - 1s 84us/step - loss: 0.4383 - acc: 0.7957
Epoch 4/10
7200/7200 [==============================] - 1s 84us/step - loss: 0.4353 - acc: 0.7957
Epoch 5/10
7200/7200 [=======

...

7200/7200 [==============================] - 1s 168us/step - loss: 0.4331 - acc: 0.7937
Epoch 11/50
7200/7200 [==============================] - 1s 169us/step - loss: 0.4339 - acc: 0.7937
Epoch 12/50
7200/7200 [==============================] - 1s 169us/step - loss: 0.4315 - acc: 0.7937
Epoch 13/50
7200/7200 [==============================] - 1s 169us/step - loss: 0.4304 - acc: 0.8065
Epoch 14/50
7200/7200 [==============================] - 1s 169us/step - loss: 0.4308 - acc: 0.8190
Epoch 15/50
7200/7200 [==============================] - 1s 171us/step - loss: 0.4285 - acc: 0.8199
Epoch 16/50
7200/7200 [==============================] - 2s 209us/step - loss: 0.4293 - acc: 0.8197
Epoch 17/50
7200/7200 [==============================] - 1s 189us/step - loss: 0.4302 - acc: 0.8222
Epoch 18/50
7200/7200 [==============================] - 1s 176us/step - loss: 0.4274 - acc: 0.8250
Epoch 19/50
7200/7200 [==============================] - 2s 216us/step - loss: 0.4280 - acc: 0.8246
Epoch 20/50


7200/7200 [==============================] - 1s 175us/step - loss: 0.4210 - acc: 0.8322
Epoch 43/50
7200/7200 [==============================] - 1s 170us/step - loss: 0.4216 - acc: 0.8333
Epoch 44/50
7200/7200 [==============================] - 1s 179us/step - loss: 0.4209 - acc: 0.8346
Epoch 45/50
7200/7200 [==============================] - 1s 179us/step - loss: 0.4232 - acc: 0.8326
Epoch 46/50
7200/7200 [==============================] - 1s 174us/step - loss: 0.4215 - acc: 0.8322
Epoch 47/50
7200/7200 [==============================] - 1s 180us/step - loss: 0.4217 - acc: 0.8332
Epoch 48/50
7200/7200 [==============================] - 1s 168us/step - loss: 0.4226 - acc: 0.8324
Epoch 49/50
7200/7200 [==============================] - 1s 167us/step - loss: 0.4226 - acc: 0.8311
Epoch 50/50
7200/7200 [==============================] - 1s 167us/step - loss: 0.4239 - acc: 0.8301
Epoch 1/50
7200/7200 [==============================] - 15s 2ms/step - loss: 0.6259 - acc: 0.7957
Epoch 2/50
720

7200/7200 [==============================] - 1s 180us/step - loss: 0.4242 - acc: 0.8268
Epoch 25/50
7200/7200 [==============================] - 1s 182us/step - loss: 0.4241 - acc: 0.8278
Epoch 26/50
7200/7200 [==============================] - 1s 184us/step - loss: 0.4238 - acc: 0.8251
Epoch 27/50
7200/7200 [==============================] - 1s 180us/step - loss: 0.4256 - acc: 0.8286
Epoch 28/50
7200/7200 [==============================] - 1s 185us/step - loss: 0.4192 - acc: 0.8271
Epoch 29/50
7200/7200 [==============================] - 1s 176us/step - loss: 0.4236 - acc: 0.8292
Epoch 30/50
7200/7200 [==============================] - 1s 175us/step - loss: 0.4203 - acc: 0.8294
Epoch 31/50
7200/7200 [==============================] - 1s 192us/step - loss: 0.4238 - acc: 0.8286
Epoch 32/50
7200/7200 [==============================] - 1s 193us/step - loss: 0.4239 - acc: 0.8282
Epoch 33/50
7200/7200 [==============================] - 1s 204us/step - loss: 0.4222 - acc: 0.8278
Epoch 34/50


7200/7200 [==============================] - 1s 171us/step - loss: 0.4346 - acc: 0.7961
Epoch 7/50
7200/7200 [==============================] - 1s 172us/step - loss: 0.4327 - acc: 0.7961
Epoch 8/50
7200/7200 [==============================] - 1s 170us/step - loss: 0.4326 - acc: 0.7961
Epoch 9/50
7200/7200 [==============================] - 1s 172us/step - loss: 0.4284 - acc: 0.7961
Epoch 10/50
7200/7200 [==============================] - 1s 172us/step - loss: 0.4294 - acc: 0.7961
Epoch 11/50
7200/7200 [==============================] - 1s 171us/step - loss: 0.4305 - acc: 0.7961
Epoch 12/50
7200/7200 [==============================] - 1s 176us/step - loss: 0.4316 - acc: 0.7961
Epoch 13/50
7200/7200 [==============================] - 1s 172us/step - loss: 0.4286 - acc: 0.7961
Epoch 14/50
7200/7200 [==============================] - 1s 170us/step - loss: 0.4312 - acc: 0.7967
Epoch 15/50
7200/7200 [==============================] - 1s 171us/step - loss: 0.4279 - acc: 0.8099
Epoch 16/50
720

8000/8000 [==============================] - 2s 310us/step - loss: 0.4226 - acc: 0.8292
Epoch 39/50
8000/8000 [==============================] - 2s 306us/step - loss: 0.4228 - acc: 0.8292
Epoch 40/50
8000/8000 [==============================] - 2s 308us/step - loss: 0.4202 - acc: 0.8277
Epoch 41/50
8000/8000 [==============================] - 2s 308us/step - loss: 0.4226 - acc: 0.8302
Epoch 42/50
8000/8000 [==============================] - 2s 293us/step - loss: 0.4216 - acc: 0.8306
Epoch 43/50
8000/8000 [==============================] - 3s 315us/step - loss: 0.4236 - acc: 0.8294
Epoch 44/50
8000/8000 [==============================] - 2s 290us/step - loss: 0.4248 - acc: 0.8281
Epoch 45/50
8000/8000 [==============================] - 2s 308us/step - loss: 0.4225 - acc: 0.8297
Epoch 46/50
8000/8000 [==============================] - 2s 311us/step - loss: 0.4223 - acc: 0.8284
Epoch 47/50
8000/8000 [==============================] - 2s 293us/step - loss: 0.4187 - acc: 0.8291
Epoch 48/50


- Melhores parâmetros encontrados e sua acurácia

In [32]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'epochs': 50, 'optimizer': 'rmsprop'}
0.8365
